## Instruction
In this notebook, We use data which I donwload from Tushare to make up some alpha factors and risk factors.
1. Load data from Tushare time range bettwen 2017.1 - 2023.3.
2. Calculate portfolio risk by PCA and save idiosynchritic values view a factor
3. Make up some factors like we did in P4 project.
5. Evaluate factor returns in 5D, 20D, 60D, 120D. and seperate factors into two parts, risk factors and alpha factors.

## Load Data

In [1]:
import pandas as pd 
import numpy as np
from tqdm import tqdm

from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 6]

import warnings
warnings.filterwarnings('ignore')

In [2]:
#universe_raw = pd.read_csv('raw_20170103_20230317.csv').iloc[:,1:]
#fundamental_df = pd.read_csv('fundamental_20170103_20230317.csv').iloc[:,1:]
universe_raw = pd.read_csv('tmp.csv').iloc[:,1:]

In [3]:
# fill null to 0
universe = universe_raw.copy(deep=True)
universe['pb'] = np.where(universe_raw['pb'].isnull(), 100., universe_raw['pb'])
universe['dt_eps'] = np.where(universe['dt_eps'].isnull(), 0., universe['dt_eps'])
universe['dt_eps_yoy'] = np.where(universe['dt_eps_yoy'].isnull(), 0., universe['dt_eps_yoy'])

## Data Explain
  Some data columns about fundamental indicators below
  - 'cfps','revenue_ps', 'quick_ratio',  每股现金流，每股营业收入，速冻比率
  - 'dt_eps','basic_eps_yoy','dt_eps_yoy',  每股收益
  - 'bps','bps_yoy',  每股净资产
  - 'extra_item','profit_dedt', 扣非，扣非净利润
  - 'roe_dt','q_dt_roe','roe_yoy',  净资产收益
  - 'capital_rese_ps','surplus_rese_ps',  每股资本公积，每股公积盈余 
  - 'gross_margin','interestdebt','ca_to_assets', 毛利，带息债务, 流动资产/总资产
  - 'ebt_yoy','roe_yoy','or_yoy','equity_yoy' 总利润增长，净资产收益增长，营业收入增长，净资产增长

### Overnight Return
This factor we had did in P4, 

We calculate factor here, and add sma indicators later.

$factor=(open_{today}-close_{yesterday})\div close_{yesterday}$

In [4]:
class CloseToOpen(pd.DataFrame):
    """
        Overnight Return Factor Constructor
    """
    def __init__(self,data):
        super(CloseToOpen, self).__init__(data)
        self.df = self

    def calculate(self):
        '''
        add open-close as a column named close_to_return
        :return: dataframe
        '''
        unique_ticker = self.df.ts_code.unique()
        tmp_df = pd.DataFrame()
        for ts_code in tqdm(unique_ticker, desc='close_to_open'):
            stock = self.df.loc[self.df.ts_code == ts_code][["ts_code", "trade_date", "open", "close"]]
            stock['alpha_close2open'] = (stock['open'].shift(-1).fillna(method='ffill') - stock['close'])/stock['close']
            tmp_df = tmp_df.append(stock, ignore_index=True)
        self.df = self.df.merge(tmp_df[["ts_code", "trade_date", "alpha_close2open"]], on=["ts_code", "trade_date"], how="inner")
        self.df['date'] = pd.to_datetime(self.df['trade_date'], format='%Y%m%d')
        self.df = self.df.set_index(['date']).sort_values(by=['date'])
        return self.df
    

universe = CloseToOpen(universe).calculate()

close_to_open: 100%|████████████████████████████████████████████████████████████████| 122/122 [00:00<00:00, 285.14it/s]


## Add Technology Indicators

In [5]:
# add volume
universe['volume'] = universe['amount']/universe['close']
universe = universe.drop_duplicates(['trade_date','ts_code'])

In [6]:
import stockstats

class IndicatorHelper(pd.DataFrame):
    """
        add indicators to dataframe
    """

    def __init__(self, data):
        super(IndicatorHelper, self).__init__(data)

        self.stocks = stockstats.StockDataFrame.retype(data.copy())
        self.df = self

    def add_technical_indicator(self, tech_indicator_list):
        """
        calculate technical indicators
        use stockstats package to add technical inidactors
        :param ticker: (df) pandas dataframe
        :param tech_indeicator_list list
        :return: (df) pandas dataframe
        """
        unique_ticker = self.df.ts_code.unique()

        indicator_df = pd.DataFrame()
        for i in tqdm(range(len(unique_ticker)), desc='add tech indicators'):
            temp_indicator = self.stocks[self.stocks.ts_code == unique_ticker[i]]
            temp_indicator = temp_indicator[tech_indicator_list + ['ts_code','trade_date']]
            indicator_df = indicator_df.append(temp_indicator, ignore_index=True)
        
        self.df = self.df.merge(indicator_df, on=["ts_code", "trade_date"], how="inner")
        self.df['date'] = pd.to_datetime(self.df['trade_date'],format='%Y%m%d')
        self.df = self.df.set_index(['date']).sort_values(by=['date']).drop_duplicates(['trade_date','ts_code'])
        return self.df
    

In [7]:
# add tech indicators
tech_indicator_list = [
                       'supertrend','close_5_sma','close_20_sma','close_60_sma',
                       'log-ret','atr_5','cci_6', 'vwma_2','vwma_25',
                       'close_10_kama_2_30', 'close_10_kama_5_30','close_2_kama',
                       'alpha_close2open_5_sma', 'alpha_close2open_20_sma'
                      ] 
universe = IndicatorHelper(universe).add_technical_indicator(tech_indicator_list)
universe['alpha_close2open_20_sma'] = -universe['alpha_close2open_20_sma']
# 'close_2_kama_20_mstd'

add tech indicators: 100%|███████████████████████████████████████████████████████████| 122/122 [00:18<00:00,  6.77it/s]


## Constructor Factors Based on Indicators

### Supertrend Factors
This factor based on supertrend and close 5 days sma

In [8]:
universe['alpha_supertrend'] = universe['close_5_sma'] - universe['supertrend']

### CCI Factors
This factor based on cci and atr 5 days window

In [9]:
universe['alpha_cci'] = -np.where(universe['cci_6']>200, (universe['cci_6']-200)*universe['atr_5'], \
                      np.where(universe['cci_6']<-200, (universe['cci_6']+200)*universe['atr_5'], universe['atr_5']*30))

### KAMA Factors

In [10]:
# add KAMA alpha factor
def KAMA_filter(df):    
    unique_stocks = df.ts_code.unique()
    all_df = pd.DataFrame()
    for ts_code in tqdm(unique_stocks, desc='kama filter'):
        tmp = df.loc[df.ts_code == ts_code]
        tmp['close_2_kama'] = tmp['close_2_kama'].fillna(method='ffill')
        tmp['kama_filter'] = tmp['close_2_kama'].rolling(window=20).std().fillna(method='bfill') * 0.6
        tmp['kama_prior'] = tmp['close_2_kama'].shift(-5).fillna(method='ffill')
        tmp['alpha_kama'] = (tmp['close_10_kama_2_30'] - tmp['close_10_kama_5_30'])-(tmp['close_2_kama'] - tmp['kama_prior'] - tmp['kama_filter'])
        #tmp['alpha_kama'] = -(tmp['close_2_kama'] - tmp['kama_prior'] - tmp['kama_filter'])
        all_df = all_df.append(tmp[['ts_code','trade_date','alpha_kama']], ignore_index=True)
    df = df.merge(all_df, on=['ts_code','trade_date'], how='left')
    df['date'] = pd.to_datetime(df['trade_date'],format='%Y%m%d')
    df = df.set_index(['date']).sort_values(by=['date'])
    return df

universe = KAMA_filter(universe)

kama filter: 100%|██████████████████████████████████████████████████████████████████| 122/122 [00:00<00:00, 167.19it/s]


## Constructor Factors Based on Paper
### Overnight Return

In [11]:
# we had add overnight return factors

### Winner And Loser
This factor we also did in P4. It express a ticker how to reach a return in a spicific period time 

We use a time window as T, and regression d and v $return = T*d + T^2*v$  => $factor=d*v$ 

In [336]:
from statsmodels.formula.api import ols

class WinnerAndLoser(pd.DataFrame):
    """
        Winner and Loser Factor Constructor
    """
    def __init__(self,data, win_length=20):
        super(WinnerAndLoser, self).__init__(data)
        self.df = self
        self.win_lenth = win_length

    def _regression(self, data):
        df = pd.DataFrame(data, columns=['log-ret'])
        df['acc_ret'] = df['log-ret'].cumsum()
        df['t_dir'] = np.arange(self.win_lenth)+1
        df['t_velocity'] = df['t_dir'] ** 2
        regression = ols(formula='acc_ret ~ 0 + t_dir + t_velocity', data=df)
        model = regression.fit()
        data['alpha_winlos'] = -model.params.t_dir * model.params.t_velocity
        return  data['alpha_winlos']


    def calculate(self):
        '''
        convert time to value
        regress return to get mu and beta each time
        add facotor mu*beta to colomns
        :return: dataframe
        '''
        tickers = self.df.ts_code.unique()
        factor_df = pd.DataFrame()
        for ticker in tqdm(tickers, desc='winner and loser'):
            tmp_df = self.df.loc[self.df.ts_code == ticker][['trade_date', 'ts_code', 'log-ret']]
            tmp_df['alpha_winlos'] = tmp_df['log-ret'].rolling(self.win_lenth).apply(self._regression)
            tmp_df['alpha_winlos'].fillna(method='bfill',inplace=True)
            factor_df = factor_df.append(tmp_df, ignore_index=True)
        self.df = self.df.merge(factor_df[["ts_code", "trade_date", "alpha_winlos"]], on=["ts_code", "trade_date"], how="left")
        self.df['date'] = pd.to_datetime(self.df['trade_date'],format='%Y%m%d')
        self.df = self.df.set_index(['date']).sort_values(by=['date'])
        return  self.df
    
    
test = universe.loc[universe.ts_code=='603538.SH']
#test = universe.loc[universe.ts_code=='002038.SZ']
test = WinnerAndLoser(test).calculate()
#universe = WinnerAndLoser(universe).calculate()

winner and loser: 100%|███████████████████████████| 1/1 [00:14<00:00, 14.97s/it]


### Skew And Momentum
This factor we also did in P4. It express minority and majority sentiment of investor how to impact on market.

We calculate skew and median of log-return distribution in a period time, the skew view as marjority sentiment and median can view as minority sentiment.

$factor = abs(skew) * median * volume\_ratio$

In [11]:
class SkewandMomentum(pd.DataFrame):
    """
        Expected Skewness and Momentum Factor Constructor
    """
    def __init__(self,data, win_length=10):
        super(SkewandMomentum, self).__init__(data)
        self.df = self
        self.win_length = win_length

    def calculate(self):
        '''
        convert time to value
        regress return to get mu and beta each time
        add facotor mu*beta to colomns
        :return: dataframe
        '''
        def calculate_factor(data):
            return abs(data.skew()) * data.median()
        
        tmp_df = pd.DataFrame()
        for stock_tuple in tqdm(self.groupby('ts_code'), desc='skew and momentum'):
            stock = stock_tuple[1]
            stock['alpha_skew2sentiment'] = stock['log-ret'].rolling(self.win_length).apply(calculate_factor)
            stock['alpha_skew2sentiment'] = stock['alpha_skew2sentiment'].fillna(method='bfill') * stock['volume_ratio']
            tmp_df = tmp_df.append(stock,ignore_index=True)
        self.df = self.df.merge(tmp_df[["ts_code", "trade_date", "alpha_skew2sentiment"]], on=["ts_code", "trade_date"], how="left")
        self.df['date'] = pd.to_datetime(self.df['trade_date'],format='%Y%m%d')
        self.df = self.df.set_index(['date']).sort_values(by=['date'])
        return self.df


#test = universe.loc[universe.ts_code=='603538.SH']
#test = universe.loc[universe.ts_code=='002038.SZ']
#test = SkewandMomentum(test).calculate()
universe = SkewandMomentum(universe).calculate()

skew and momentum: 100%|█████████████████████████████████████████████████████████████| 122/122 [00:08<00:00, 14.93it/s]


## Fundamental Factor
This factor based on ticker fundamentals, it usually take a long period time to archive return. So called take a long line to catch a big fish! The factor we define as:

$ factor1 = mean(volume\_5\_windows) - std(volume\_25\_windows) * ((6 \div pe) + (0.5 \div pb)) * dt\_eps$

$ factor2 =  abs(dt\_eps) * dt\_eps\_yoy * type\_value \div 30 $

$ alpha\_factor = factor1 + 0.3 * factor2 $

- volume: trade volume
- pb: profit div balance
- pe: profit div net balance
- type_value: fundamentals prereport levels from -3 to 3
- dt_eps: prifit each share of stock
- dt_eps_yoy: profit increase percent of dt_esp

In [13]:
from scipy.stats import zscore
def fundamentals_alpha_fundamental(df):
    all_df = pd.DataFrame()
    for ts_code in tqdm(df.ts_code.unique(), desc='fundamental factor'):
        tmp = df.loc[df.ts_code==ts_code]
        tmp['alpha_fundamental1'] = (tmp['vwma_2'] - tmp['vwma_25'])* (6/tmp['pe'] + 0.5/tmp['pb'])*(tmp['dt_eps']**2)
        tmp['alpha_fundamental2'] = np.where(tmp['dt_eps']>0, tmp['dt_eps'], 0.) * tmp['dt_eps_yoy'] / 100  * np.where(tmp['type_value']<0., 0.5, 2)
        tmp[['alpha_fundamental1','alpha_fundamental2']] = tmp[['alpha_fundamental1','alpha_fundamental2']].apply(zscore)
        tmp['alpha_fundamental'] = tmp['alpha_fundamental1'] + tmp['alpha_fundamental2'] 
        tmp['alpha_fundamental'].fillna(-1e-4,inplace = True)
        all_df = all_df.append(tmp, ignore_index=True)
        
    df = df.merge(all_df[['ts_code','trade_date','alpha_fundamental']], on=['ts_code','trade_date'], how='left')
    df['date'] = pd.to_datetime(df['trade_date'],format='%Y%m%d')
    df = df.set_index(['date']).sort_values(by=['date'])
    return df
   
#universe = universe.drop(columns=['alpha_fundamental'])
universe = fundamentals_alpha_fundamental(universe)

fundamental factor: 100%|████████████████████████████████████████████████████████████| 122/122 [00:01<00:00, 77.27it/s]


## Ticker Pool
After create factors we filte data by each day more in detail. 

We do this step after calculate factors in case of some tickers add in our portfolio calculate factor by historical data which not exists.

In real trade with this model factor use, download data each day, if you ensure add new tickers into portfolio, ensure get data 2 month ago at least, then calculate factos.

In [15]:
# remove tickers by each day not exist history
def remove_tickers(df, exist_ticker_list):
    diff_df = df.loc[df.ts_code.isin(exist_ticker_list)==False]
    if diff_df.empty == False:
        # remove tickers not exist over 3month from day
        diff1 = (pd.to_datetime(diff_df['trade_date'],format='%Y%m%d')\
                -pd.to_datetime(diff_df['list_date'],format='%Y%m%d')).apply(lambda x: x.days) < 90
        # remove tickers pe > 80
        diff2 = diff_df.pe > 80
        # remove tickers pb > 10
        diff3 = diff_df.pb > 10
        # remove fundamental bad
        diff4 = diff_df['type_value']<-1
        # dt_eps <0 means profit is below 0.3 , remove it.
        diff5 = diff_df['dt_eps']<0.3
        # total_mv>50 billion
        diff6 = diff_df['total_mv']>9000000
        # get remove df
        diff_df = diff_df.loc[diff1| diff2| diff3| diff4| diff5| diff6]
        # get rest data
        df = df.loc[df.ts_code.isin(diff_df.ts_code)==False]
    return df

# clean tickers pool day by day
calendar = universe.trade_date.unique()
universe_raw = pd.DataFrame()
for dt in tqdm(calendar, desc='filter tikers'):
    tmp = universe.loc[universe['trade_date']==dt]
    if universe_raw.empty:
        tmp = remove_tickers(tmp, [])
    else:
        tmp = remove_tickers(tmp, universe_raw.ts_code)
    universe_raw = universe_raw.append(tmp, ignore_index=True)

universe_raw['date'] = pd.to_datetime(universe_raw['trade_date'], format='%Y%m%d')
universe_raw = universe_raw.set_index(['date']).sort_values(by=['date'])
print(universe_raw.shape, len(universe_raw.ts_code.unique()))
universe_raw.head()

filter tikers: 100%|████████████████████████████████████████████████████████████████| 243/243 [00:01<00:00, 154.76it/s]

(10607, 77) 56


,ts_code,trade_date,turnover_rate,volume_ratio,pe,pb,total_share,free_share,total_mv,circ_mv,...,close_10_kama_2_30,close_10_kama_5_30,close_2_kama,alpha_close2open_5_sma,alpha_close2open_20_sma,alpha_supertrend,alpha_cci,alpha_kama,alpha_skew2sentiment,alpha_fundamental
date,,,,,,,,,,,,,,,,,,,,,
2018-01-02,300183.SZ,20180102,1.4832,0.72,25.4897,3.4960,47030.9857,20410.8707,9.086386e+05,5.090988e+05,...,17.92257,17.92257,17.92257,-0.001035,0.001035,-0.946240,-10.5756,0.142329,0.000295,-0.589884
2018-01-02,002435.SZ,20180102,0.3413,1.26,52.9549,1.6279,49077.6795,12274.2144,8.480623e+05,3.174855e+05,...,6.44209,6.44209,6.44209,0.005208,-0.005208,-0.184530,-1.6776,0.117876,0.002603,1.814644
2018-01-02,300533.SZ,20180102,1.0670,0.86,32.6185,3.4140,10000.0000,3065.6779,5.071000e+05,1.818912e+05,...,32.29039,32.29039,32.29039,-0.000592,0.000592,-0.926490,-10.5066,0.071707,0.001470,-0.786528
2018-01-02,002020.SZ,20180102,1.8545,0.49,39.8719,2.3107,73612.4472,43109.0368,8.450709e+05,5.584441e+05,...,8.46846,8.46846,8.46846,-0.002613,0.002613,-0.461025,-4.2045,-0.213766,-0.000644,1.236279
2018-01-02,002589.SZ,20180102,0.6919,0.62,34.1563,2.6826,150471.0471,76256.2164,2.017817e+06,1.210350e+06,...,13.16946,13.16946,13.16946,0.005220,-0.005220,-0.584350,-6.1872,0.094116,-0.001022,1.856420


In [16]:
# filter from 20170405
# filte data columns as we use to next step
field_list = [
       'ts_code', 'trade_date', 'turnover_rate', 'open', 'close', 'log-ret', 'pe', 'pb','amount', 'total_mv', 'circ_mv', 
       'type', 'type_value', 'name', 'industry',
       'issue_price', 'issue_amount', 'cfps','revenue_ps',
       'quick_ratio', 'dt_eps', 'basic_eps_yoy', 'dt_eps_yoy',
       'bps', 'bps_yoy', 'profit_dedt', 'roe_dt', 'q_dt_roe',
       'roe_yoy', 'capital_rese_ps', 'surplus_rese_ps', 'gross_margin',
       'interestdebt', 'ca_to_assets', 'ebt_yoy', 'or_yoy', 'equity_yoy', 
       'alpha_cci', 'alpha_kama', 'alpha_close2open',
       'alpha_close2open_5_sma', 'alpha_close2open_20_sma',
       'alpha_skew2sentiment', 'alpha_fundamental']
universe_raw = universe_raw.loc[universe_raw['trade_date']>=20170405]
universe_factors = universe_raw[field_list]
universe_factors.head()

,ts_code,trade_date,turnover_rate,open,close,log-ret,pe,pb,amount,total_mv,...,ebt_yoy,or_yoy,equity_yoy,alpha_cci,alpha_kama,alpha_close2open,alpha_close2open_5_sma,alpha_close2open_20_sma,alpha_skew2sentiment,alpha_fundamental
date,,,,,,,,,,,,,,,,,,,,,
2018-01-02,300183.SZ,20180102,1.4832,17.82980,17.92257,0.0,25.4897,3.4960,75186.17,9.086386e+05,...,-33.6280,-11.0714,8.0217,-10.5756,0.142329,-0.001035,-0.001035,0.001035,0.000295,-0.589884
2018-01-02,002435.SZ,20180102,0.3413,6.48682,6.44209,0.0,52.9549,1.6279,10849.69,8.480623e+05,...,6.0340,27.3816,498.6090,-1.6776,0.117876,0.005208,0.005208,-0.005208,0.002603,1.814644
2018-01-02,300533.SZ,20180102,1.0670,32.34133,32.29039,0.0,32.6185,3.4140,19345.40,5.071000e+05,...,-43.8649,-21.4672,2.5131,-10.5066,0.071707,-0.000592,-0.000592,0.000592,0.001470,-0.786528
2018-01-02,002020.SZ,20180102,1.8545,8.54223,8.46846,0.0,39.8719,2.3107,104010.18,8.450709e+05,...,23.9800,14.5072,54.5084,-4.2045,-0.213766,-0.002613,-0.002613,0.002613,-0.000644,1.236279
2018-01-02,002589.SZ,20180102,0.6919,13.14982,13.16946,0.0,34.1563,2.6826,83495.11,2.017817e+06,...,123.9639,52.9144,12.9861,-6.1872,0.094116,0.005220,0.005220,-0.005220,-0.001022,1.856420


## Calculate idiosynchritic risk
We calculate idiosynchritic risk of each tickers by each day

$residual\_ticker = \sum_{i = ticker}tiker\_return\_i - \frac{industry\_marketcap}{total\_marketcap} * industry\_index\_return$

In [17]:
# flag industry to 1 and 2, 1 to medicine_1000 index and rest to internet index
def factors_from_names(n, name):
    return list(filter(lambda x: name in x, n))

universe_factors['industry_flag'] = np.where(universe_factors['industry'].isin(factors_from_names(universe_factors['industry'], '药')), 1, 2)

In [ ]:
# get index return
def cal_index_return(df, index_df):
    marketcap_total = df['total_mv'].sum()
    industries = df.industry_flag.unique()
    for flag in industries:
        marketcap_industry = df.loc[df['industry_flag'] == flag]['total_mv'].sum()
        index_return_val = (marketcap_industry / marketcap_total) * index_df[flag]
        df.loc[df['industry_flag'] == flag]['index_return'] == index_return_val.values
    return df

In [19]:
## save tmp data
# fundamental_df.to_csv('fundamental_20170103_20230317.csv')
# universe_raw.to_csv('raw_20170103_20230317.csv')
# universe_factors.to_csv('factors_20170103_20230317.csv')

## load universe_factors
# universe_factors = pd.read_csv('tmp_factor.csv')
# universe_factors['date'] = pd.to_datetime(universe_factors['date'],format='%Y-%m-%d')
# universe_factors.set_index(['date'],inplace=True)

print(len(universe_factors.ts_code.unique()), len(universe.ts_code.unique()))
#universe_factors.loc[universe_factors.ts_code=='603538.SH'][['total_mv','dt_eps','dt_eps_yoy','type_value']]

56 122


In [20]:
from scipy.stats import zscore
tmp = universe_factors.loc[universe_factors.ts_code== '002649.SZ']
#tmp = factor_df.loc[factor_df.ts_code== '002649.SZ']
tmp[['alpha_kama','alpha_fundamental','close']] = tmp[['alpha_kama','alpha_fundamental','close']].apply(zscore,ddof=1).fillna(0.)

#tmp['alpha'] = tmp['alpha'].rank(method='min', pct=True)
tmp[['alpha_kama','alpha_fundamental']] = tmp[['alpha_kama','alpha_fundamental']]
#tmp['alpha_fundamental'] = tmp['alpha_fundamental']
tmp['close'] = tmp['close']
tmp[['close','alpha_fundamental', 'alpha_kama']].plot(grid=True)
#tmp[['close','alpha_fundamental']].plot(grid=True)
#tmp[['vwma_5','vwma_25','close','vr_6']].plot(subplots=True, grid=True, figsize=(8, 10))
#tmp[['vwma_5','vwma_25','close']].plot(grid=True)